>MACHINE LEARNING


> 22MCA0394


> ASHISH PATEL


1) Develop a RNN model for analysis the sentiment of the tweets using “Sentiment140 dataset”
Dataset: https://www.kaggle.com/datasets/kazanova/sentiment140

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/archive.zip

Archive:  /content/drive/MyDrive/archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [4]:
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',
                   encoding='latin', names = ['polarity','id','date','query','user','text'])
data.head()

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [5]:
print("Dataset shape:", data.shape)

Dataset shape: (1600000, 6)


In [6]:
data['polarity'].unique()

array([0, 4])

In [7]:
data['polarity'] = data['polarity'].replace(4,1)
data.head(1600000)

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [8]:
data.describe()

,polarity,id
count,1600000.0,1.600000e+06
mean,0.5,1.998818e+09
std,0.5,1.935761e+08
min,0.0,1.467810e+09
25%,0.0,1.956916e+09
50%,0.5,2.002102e+09
75%,1.0,2.177059e+09
max,1.0,2.329206e+09


In [9]:
positives = data['polarity'][data.polarity == 1 ]
negatives = data['polarity'][data.polarity == 0 ]

print('Total length of the data is:         {}'.format(data.shape[0]))
print('No. of positve tagged sentences is:  {}'.format(len(positives)))
print('No. of negative tagged sentences is: {}'.format(len(negatives)))

Total length of the data is:         1600000
No. of positve tagged sentences is:  800000
No. of negative tagged sentences is: 800000


In [10]:
data.drop(['id','date','query','user'], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   polarity  1600000 non-null  int64 
 1   text      1600000 non-null  object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [11]:
data.head(10)

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


In [12]:
(data.isnull().sum() / len(data))*100

polarity    0.0
text        0.0
dtype: float64

In [13]:
data['text'] = data['text'].astype('str')

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stopword = set(stopwords.words('english'))
print(stopword)

{'were', 'further', 'does', 'myself', 'will', 'down', 'out', 'only', "weren't", 'as', 'are', 'from', "doesn't", 'your', 'll', 'if', 'same', 'their', 'once', 'its', 'own', 'not', 'hasn', 'his', 'ma', 'these', 'after', "you'd", 'itself', "it's", 'her', 'who', 'the', 'off', 'mightn', 'them', 'been', 'such', 'should', 'do', 're', 'himself', 'shan', 'y', 'needn', 'before', 'both', 'up', 'hers', 'very', "won't", 'we', 'yourself', "haven't", 'no', 'during', 'did', 'which', 'why', 'theirs', 'while', 'has', 'so', 'doesn', "wouldn't", 'weren', 'on', "shouldn't", "she's", 'am', 'here', 'she', 'mustn', 'i', "hasn't", 'or', 'what', "couldn't", 'and', 'don', 'hadn', "that'll", 'most', "didn't", 'all', 'those', 'few', "you've", 'by', 'me', "isn't", 'whom', 'some', 'until', 'against', 'there', 'for', 'over', "you're", 'had', "you'll", 'between', 'yours', 'at', 'isn', 'about', 'can', 'm', 'just', 'wouldn', 'when', 'below', 'nor', 't', 'couldn', 's', 'didn', 'it', 'under', "hadn't", 'than', 'themselves'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
import re
import string

urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'
def process_tweets(tweet):
  # Lower Casing
    tweet = tweet.lower()
    tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [16]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
data['processed_tweets'] = data['text'].apply(lambda x: process_tweets(x))
print('Text Preprocessing complete.')

Text Preprocessing complete.


In [18]:
data.head(10)

,polarity,text,processed_tweets
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww thats bummer shoulda got david...
1,0,is upset that he can't update his Facebook by ...,upset cant update facebook texting might cry r...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dived many time ball managed save 50 ...
3,0,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving im mad cant see
5,0,@Kwesidei not the whole crew,kwesidei whole crew
6,0,Need a hug,eed hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,loltrish hey long time see yes rain bit bit lo...
8,0,@Tatiana_K nope they didn't have it,tatianak nope didnt
9,0,@twittera que me muera ?,twittera que muera


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = data['processed_tweets'].values
y = data['polarity'].values
print(X.shape)
print(y.shape)

vector = TfidfVectorizer(sublinear_tf=True)
X = vector.fit_transform(X)
print(f'Vector fitted.')

print(X.shape)
print(y.shape)

(1600000,)
(1600000,)
Vector fitted.
(1600000, 762358)
(1600000,)


In [20]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import regularizers

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.processed_tweets)
sequences = tokenizer.texts_to_sequences(data.processed_tweets)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  759 1865    2]
 [   0    0    0 ...   11  187 1283]
 [   0    0    0 ...  368    6 3132]
 ...
 [   0    0    0 ...  123  504 1746]
 [   0    0    0 ...  415 4997   12]
 [   0    0    0 ...    0    0  458]]


In [21]:
X_train, X_test, y_train, y_test = train_test_split(tweets, data.polarity.values,
                                                    test_size = 0.2, random_state=101)

In [22]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print()
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (1280000, 200)
y_train (1280000,)

X_test (320000, 200)
y_test (320000,)


In [23]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
model = Sequential()
model.add(layers.Embedding(max_words, 128))
model.add(layers.LSTM(64,dropout=0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         640000    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 690593 (2.63 MB)
Trainable params: 690593 (2.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.fit(X_train,y_train,epochs=10,batch_size= 128, validation_split=0.2,
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
8000/8000 [==============================] - 142s 17ms/step - loss: 0.4787 - accuracy: 0.7683 - val_loss: 0.4617 - val_accuracy: 0.7795
Epoch 2/10
8000/8000 [==============================] - 97s 12ms/step - loss: 0.4572 - accuracy: 0.7818 - val_loss: 0.4533 - val_accuracy: 0.7858
Epoch 3/10
8000/8000 [==============================] - 95s 12ms/step - loss: 0.4488 - accuracy: 0.7872 - val_loss: 0.4514 - val_accuracy: 0.7870
Epoch 4/10
8000/8000 [==============================] - 95s 12ms/step - loss: 0.4426 - accuracy: 0.7906 - val_loss: 0.4494 - val_accuracy: 0.7876
Epoch 5/10
8000/8000 [==============================] - 94s 12ms/step - loss: 0.4379 - accuracy: 0.7932 - val_loss: 0.4486 - val_accuracy: 0.7890
Epoch 6/10
8000/8000 [==============================] - 96s 12ms/step - loss: 0.4341 - accuracy: 0.7956 - val_loss: 0.4477 - val_accuracy: 0.7899
Epoch 7/10
8000/8000 [==============================] - 98s 12ms/step - loss: 0.4303 - accuracy: 0.7979 - val_loss: 0.4497 

In [25]:
accr = model.evaluate(X_test,y_test)

10000/10000 [==============================] - 51s 5ms/step - loss: 0.4513 - accuracy: 0.7873


In [26]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.451
  Accuracy: 0.787
